In [1]:
import scanpy as sc
import spatialSNV as ss

/home/liuyi/miniconda3/envs/spatialsnv/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/liuyi/miniconda3/envs/spatialsnv/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/home/liuyi/miniconda3/envs/spatialsnv/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/home/liuyi/miniconda3/envs/spatialsnv/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/home/liuyi/miniconda3/envs/spatialsnv/lib/python3.10/site-packages/anndata/utils.py:429: 

In [2]:
root_path = '/storage/liuyi/00.SNP_project/code/gigascience_update/'

In [3]:
stereo_sample_names = ['CRC-P19-T','CRC-P59-T_1','CRC-P59-T_2','CRC-P67-T','LC05-M_DU3','LC05-T_FD3']
visium_sample_names = ['dcis1','dcis2']
total_sample_names = visium_sample_names + stereo_sample_names

In [4]:
stereo_gtf = f"{root_path}/reference_forhg38/gencode.v31.chr_patch_hapl_scaff.annotation.gtf.gz"
stereo_annovar_ref = f"{root_path}/reference_forhg38/p12_ref" # bulid by annovar
visium_gtf = f"{root_path}/reference_forhg38/genes.gtf.gz"
visium_annovar_ref = f"{root_path}/reference_forhg38/10x_ref" # bulid by annovar

annovar_spe = "homo"
annovar_ref_name = "gencodev38"

In [5]:
for sample in total_sample_names:
    rna = sc.read_h5ad(f'{root_path}/rna_adata/{sample}.rna.h5ad')
    if sample in stereo_sample_names:
        snv_path = f"{root_path}/input_matrix/{sample}/{sample}_snp100_matrix"
        snv_depth_path = f"{root_path}/input_matrix/{sample}/{sample}_snp100_matrix_depth"
        gtf = stereo_gtf
        annovar_ref = stereo_annovar_ref
    else:
        snv_path = f"{root_path}/input_matrix/{sample}/{sample}_snp_matrix"
        snv_depth_path = f"{root_path}/input_matrix/{sample}/{sample}_snp_matrix_depth"
        gtf = visium_gtf
        annovar_ref = visium_annovar_ref
        
    snv = sc.read_10x_mtx(snv_path,cache = False)
    snv_depth = sc.read_10x_mtx(snv_depth_path,cache = False)
    if sample in stereo_sample_names:
        snv.obs_names = 'DNB_'+ snv.obs_names
        snv_depth.obs_names = 'DNB_'+ snv_depth.obs_names
    
    common = list(set(rna.obs_names).intersection(set(snv_depth.obs_names)))
    rna = rna[common, :].copy()
    snv_depth = snv_depth[common,:].copy()
    snv = snv[common,snv_depth.var_names].copy()
    snv.obsm['spatial'] = rna.obsm['spatial']
    sc.pp.filter_genes(snv,min_cells = 1)
    sc.pp.filter_genes(snv_depth,min_cells = 1)
    df_n_cells = snv.var['n_cells']
    df_n_cells.to_csv(f'{root_path}/otherdata_for_figure/Figure1/snv_cells/{sample}.cells.csv',index=None)

    
    snv = ss.processsnv(
        sample,
        snv,
        snv_depth,
        gtf = gtf,
        annovar_ref = annovar_ref,
        annovar_spe = annovar_spe,
        annovar_ref_name = annovar_ref_name,
        thrshold = 20,   # Filter SNV by coverage
        min_cells = 5,   # Filter SNV by spot
        outdir = f"./out/",
        annovar = "/home/liuyi/02.software/annovar/annovar/table_annovar.pl", # path to annovar
    )
    ss.normalize_with_rna(snv,rna)
    snv.write(f'{root_path}/snv_adata/{sample}.snv.h5ad')

--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
filtered out 21325 genes that are detected in less than 1 cells
filtered out 21325 genes that are detected in less than 1 cells


NOTICE: the --polish argument is set ON automatically (use --nopolish to change this behavior)
-----------------------------------------------------------------
NOTICE: Processing operation=g protocol=gencodev38

NOTICE: Running with system command <annotate_variation.pl -geneanno -buildver homo -dbtype gencodev38 -outfile ./out//dcis1.gencodev38 -exonsort -nofirstcodondel ./out//dcis1.avinput /storage/liuyi/00.SNP_project/code/gigascience_update//reference_forhg38/10x_ref>
NOTICE: Output files are written to ./out//dcis1.gencodev38.variant_function, ./out//dcis1.gencodev38.exonic_variant_function
NOTICE: Reading gene annotation from /storage/liuyi/00.SNP_project/code/gigascience_update/reference_forhg38/10x_ref/homo_gencodev38.txt ... Done with 199138 transcripts (including 99933 without coding sequence annotation) for 36601 unique genes
NOTICE: Processing next batch with 7215 unique variants in 7215 input lines
NOTICE: Reading FASTA sequences from /storage/liuyi/00.SNP_project/code/g

--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
filtered out 33125 genes that are detected in less than 1 cells
filtered out 33125 genes that are detected in less than 1 cells


NOTICE: the --polish argument is set ON automatically (use --nopolish to change this behavior)
-----------------------------------------------------------------
NOTICE: Processing operation=g protocol=gencodev38

NOTICE: Running with system command <annotate_variation.pl -geneanno -buildver homo -dbtype gencodev38 -outfile ./out//dcis2.gencodev38 -exonsort -nofirstcodondel ./out//dcis2.avinput /storage/liuyi/00.SNP_project/code/gigascience_update//reference_forhg38/10x_ref>
NOTICE: Output files are written to ./out//dcis2.gencodev38.variant_function, ./out//dcis2.gencodev38.exonic_variant_function
NOTICE: Reading gene annotation from /storage/liuyi/00.SNP_project/code/gigascience_update/reference_forhg38/10x_ref/homo_gencodev38.txt ... Done with 199138 transcripts (including 99933 without coding sequence annotation) for 36601 unique genes
NOTICE: Processing next batch with 6286 unique variants in 6286 input lines
NOTICE: Reading FASTA sequences from /storage/liuyi/00.SNP_project/code/g

--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
filtered out 11997 genes that are detected in less than 1 cells
filtered out 11997 genes that are detected in less than 1 cells
filtered out 1 genes that are detected in less than 5 cells


...

NOTICE: the --polish argument is set ON automatically (use --nopolish to change this behavior)
-----------------------------------------------------------------
NOTICE: Processing operation=g protocol=gencodev38

NOTICE: Running with system command <annotate_variation.pl -geneanno -buildver homo -dbtype gencodev38 -outfile ./out//CRC-P67-T.gencodev38 -exonsort -nofirstcodondel ./out//CRC-P67-T.avinput /storage/liuyi/00.SNP_project/code/gigascience_update//reference_forhg38/p12_ref>
NOTICE: Output files are written to ./out//CRC-P67-T.gencodev38.variant_function, ./out//CRC-P67-T.gencodev38.exonic_variant_function
NOTICE: Reading gene annotation from /storage/liuyi/00.SNP_project/code/gigascience_update/reference_forhg38/p12_ref/homo_gencodev38.txt ... Done with 247086 transcripts (including 137441 without coding sequence annotation) for 66738 unique genes
NOTICE: Processing next batch with 44912 unique variants in 44912 input lines
NOTICE: Reading FASTA sequences from /storage/liuyi/00

--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
filtered out 5171 genes that are detected in less than 1 cells
filtered out 5171 genes that are detected in less than 1 cells


NOTICE: the --polish argument is set ON automatically (use --nopolish to change this behavior)
-----------------------------------------------------------------
NOTICE: Processing operation=g protocol=gencodev38

NOTICE: Running with system command <annotate_variation.pl -geneanno -buildver homo -dbtype gencodev38 -outfile ./out//LC05-M_DU3.gencodev38 -exonsort -nofirstcodondel ./out//LC05-M_DU3.avinput /storage/liuyi/00.SNP_project/code/gigascience_update//reference_forhg38/p12_ref>
NOTICE: Output files are written to ./out//LC05-M_DU3.gencodev38.variant_function, ./out//LC05-M_DU3.gencodev38.exonic_variant_function
NOTICE: Reading gene annotation from /storage/liuyi/00.SNP_project/code/gigascience_update/reference_forhg38/p12_ref/homo_gencodev38.txt ... Done with 247086 transcripts (including 137441 without coding sequence annotation) for 66738 unique genes
NOTICE: Processing next batch with 23435 unique variants in 23435 input lines
NOTICE: Reading FASTA sequences from /storage/liuy

--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
filtered out 8408 genes that are detected in less than 1 cells
filtered out 8408 genes that are detected in less than 1 cells
filtered out 6 genes that are detected in less than 5 cells


NOTICE: the --polish argument is set ON automatically (use --nopolish to change this behavior)
-----------------------------------------------------------------
NOTICE: Processing operation=g protocol=gencodev38

NOTICE: Running with system command <annotate_variation.pl -geneanno -buildver homo -dbtype gencodev38 -outfile ./out//LC05-T_FD3.gencodev38 -exonsort -nofirstcodondel ./out//LC05-T_FD3.avinput /storage/liuyi/00.SNP_project/code/gigascience_update//reference_forhg38/p12_ref>
NOTICE: Output files are written to ./out//LC05-T_FD3.gencodev38.variant_function, ./out//LC05-T_FD3.gencodev38.exonic_variant_function
NOTICE: Reading gene annotation from /storage/liuyi/00.SNP_project/code/gigascience_update/reference_forhg38/p12_ref/homo_gencodev38.txt ... Done with 247086 transcripts (including 137441 without coding sequence annotation) for 66738 unique genes
NOTICE: Processing next batch with 57137 unique variants in 57137 input lines
NOTICE: Reading FASTA sequences from /storage/liuy

# slide_RNA

In [6]:
sample = 'slide_RNA'
rna = sc.read_h5ad(f'{root_path}/rna_adata/{sample}.rna.h5ad')
snv_path = f"{root_path}/input_matrix/{sample}/{sample}_snp_matrix"
snv_depth_path = f"{root_path}/input_matrix/{sample}/{sample}_snp_matrix_depth"

In [7]:
import pandas as pd
import anndata as ad
import numpy as np
from scipy.sparse import csr_matrix

In [10]:
snv_depth = sc.read_10x_mtx(snv_depth_path,cache=False)
snv = sc.read_10x_mtx(snv_path,cache=False)
second_column=pd.read_csv(f'{root_path}/input_matrix/{sample}/human_colon_cancer_4_rna_200102_06.bead_locations.csv')
location = second_column.set_index('barcode')
snv.obs['x'] = location['xcoord']
snv.obs['y'] = location['ycoord']
snv = snv[snv.obs_names.isin(snv.obs.dropna().index)]
snv_depth.obs['x'] = location['xcoord']
snv_depth.obs['y'] = location['ycoord']
snv_depth = snv_depth[snv_depth.obs_names.isin(snv_depth.obs.dropna().index)]

df = snv.to_df()

## bin50
df[['x', 'y']] = snv.obs[['x', 'y']].applymap(lambda x: int(x/50)*50)
df['id'] = df['x'].apply(str) + '_' + df['y'].apply(str)
matrix = df.groupby(by='id').sum()
del matrix['x']
del matrix['y']
snv = ad.AnnData(matrix)
snv.obs['x']=snv.obs.index.map(lambda x : int(x.split('_')[0]))
snv.obs['y']=snv.obs.index.map(lambda x : int(x.split('_')[1]))
snv.obsm['spatial']=np.array([snv.obs['x'],snv.obs['y']]).T
depdf = snv_depth.to_df()
depdf[['x', 'y']] = snv_depth.obs[['x', 'y']].applymap(lambda x: int(x/50)*50)
depdf['id'] = depdf['x'].apply(str) + '_' + depdf['y'].apply(str)
depmatrix = depdf.groupby(by='id').sum()
del depmatrix['x']
del depmatrix['y']
snv_depth = ad.AnnData(depmatrix)
snv_depth.obs['x']=snv_depth.obs.index.map(lambda x : int(x.split('_')[0]))
snv_depth.obs['y']=snv_depth.obs.index.map(lambda x : int(x.split('_')[1]))
snv_depth.obsm['spatial']=np.array([snv_depth.obs['x'],snv_depth.obs['y']]).T

snv.X = csr_matrix(snv.X)
snv_depth.X = csr_matrix(snv_depth.X)
common = list(set(rna.obs_names).intersection(set(snv_depth.obs_names)))
rna = rna[common, :].copy()
snv_depth = snv_depth[common,:].copy()
snv = snv[common,snv_depth.var_names].copy()
snv.obsm = rna.obsm
sc.pp.filter_genes(snv,min_cells = 1)
sc.pp.filter_genes(snv_depth,min_cells = 1)
df_n_cells = snv.var['n_cells']
df_n_cells.to_csv(f'{root_path}/otherdata_for_figure/Figure1/snv_cells/{sample}.cells.csv',index=None)
snv = ss.processsnv(
        sample,
        snv,
        snv_depth,
        gtf = gtf,
        annovar_ref = annovar_ref,
        annovar_spe = annovar_spe,
        annovar_ref_name = annovar_ref_name,
        thrshold = 20,   # Filter SNV by coverage
        min_cells = 5,   # Filter SNV by spot
        outdir = f"./out/",
        annovar = "/home/liuyi/02.software/annovar/annovar/table_annovar.pl", # path to annovar
    )
ss.normalize_with_rna(snv,rna)
snv.write(f'{root_path}/snv_adata/{sample}.snv.h5ad')

--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
filtered out 221759 genes that are detected in less than 1 cells
filtered out 221759 genes that are detected in less than 1 cells


NOTICE: the --polish argument is set ON automatically (use --nopolish to change this behavior)
-----------------------------------------------------------------
NOTICE: Processing operation=g protocol=gencodev38

NOTICE: Running with system command <annotate_variation.pl -geneanno -buildver homo -dbtype gencodev38 -outfile ./out//slide_RNA.gencodev38 -exonsort -nofirstcodondel ./out//slide_RNA.avinput /storage/liuyi/00.SNP_project/code/gigascience_update//reference_forhg38/p12_ref>
NOTICE: Output files are written to ./out//slide_RNA.gencodev38.variant_function, ./out//slide_RNA.gencodev38.exonic_variant_function
NOTICE: Reading gene annotation from /storage/liuyi/00.SNP_project/code/gigascience_update/reference_forhg38/p12_ref/homo_gencodev38.txt ... Done with 247086 transcripts (including 137441 without coding sequence annotation) for 66738 unique genes
NOTICE: Processing next batch with 1245 unique variants in 1245 input lines
NOTICE: Reading FASTA sequences from /storage/liuyi/00.S

# slide DNA

In [11]:
sample_name = 'slide_DNA'
snv_depth_path = f"{root_path}/input_matrix/{sample_name}/snv_matrix_depth"
beads = pd.read_csv(f"{root_path}/input_matrix/{sample_name}/human_colon_cancer_3_dna_191204_19.bead_locations.csv")

In [12]:
rev_compl_l = [chr(i) for i in range(128)]
rev_compl_l[ord('A')] = 'T'
rev_compl_l[ord('C')] = 'G'
rev_compl_l[ord('G')] = 'C'
rev_compl_l[ord('T')] = 'A'

def rev_compl(s):
    return ''.join(rev_compl_l[ord(c)] for c in reversed(s))

In [13]:
snv_depth = sc.read_10x_mtx(snv_depth_path,cache=True)
snv_depth = snv_depth[snv_depth.obs_names.isin(beads['barcodes'].map(rev_compl))]
snv_depth.var['SNVDepth'] = np.sum(snv_depth.X,axis = 0).reshape(-1,1)
snv_depth.obs['TotalDepth'] = np.sum(snv_depth.X,axis = 1)

... writing an h5ad cache file to speedup reading next time


/tmp/ipykernel_192401/1746673476.py:3: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  snv_depth.var['SNVDepth'] = np.sum(snv_depth.X,axis = 0).reshape(-1,1)


In [14]:
sc.pp.filter_genes(snv_depth,min_cells = 1)
df_n_cells = snv_depth.var['n_cells']
df_n_cells.to_csv(f'{root_path}/otherdata_for_figure/Figure1/snv_cells/{sample_name}.cells.csv',index=None)
thrshold = 20
sub_snv = snv_depth[:,snv_depth.var['SNVDepth'] >= thrshold].copy()
sc.pp.filter_genes(sub_snv,min_cells = 3)

filtered out 46994 genes that are detected in less than 1 cells
filtered out 4 genes that are detected in less than 3 cells


In [15]:
sub_snv.write(f'{root_path}/snv_adata/slide_DNA.snv.h5ad')